In [8]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

def DefinePISO(n):
    @si.coroutine(inputs={"PI": si.Bits(n), "SI": si.Bit, "LOAD": si.Bit})
    def PISO():
        values = bits(0, n)
        O = values[-1]
        while True:
            SI, PI, LOAD = yield O
            O = values[-1]
            if LOAD:
                values = PI
            else:
                # values = [SI] + values[:-1]
                for i in range(n - 1, 0, -1):
                    values[i] = values[i - 1]
                values[0] = SI
    return PISO
        
PISO10 = si.compile(DefinePISO(10)(), file_name="build/silica_piso.py")
with open("build/silica_piso.py", "r") as magma_file:
    print(magma_file.read())

from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width):
    if "one-hot" == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
                            wire(and_.I1[j], io.S[i])
                    else:
   

In [2]:
from magma.bitutils import int2seq
piso = DefinePISO(10)()
message = [0xDE, 0xAD, 0xBE, 0xEF]

@si.coroutine
def inputs_generator(message):
    while True:
        for byte in message:
            PI = [bool(x) for x in [0] + int2seq(byte) + [1]]
            SI = False
            LOAD = True
            yield PI, SI, LOAD
            for i in range(10):
                LOAD = False
                PI = [bool(x) for x in int2seq(0xFF, 10)]
                yield PI, SI, LOAD

inputs = inputs_generator(message)

for i in range(len(message)):
    expected_outputs = inputs.PI[:]
    expected_state = inputs.PI[:]
    piso.send((inputs.PI, inputs.SI, inputs.LOAD))
    # print(f"PI={inputs.PI}, SI={inputs.SI}, LOAD={inputs.LOAD}, O={piso.O}, values={piso.values}")
    next(inputs)
    actual_outputs = []
    for i in range(10):
        expected_state = [inputs.SI] + expected_state[:-1]
        piso.send((inputs.PI, inputs.SI, inputs.LOAD))
        assert piso.values == expected_state
        actual_outputs.insert(0, piso.O)
        # print(f"PI={inputs.PI}, SI={inputs.SI}, LOAD={inputs.LOAD}, O={piso.O}, values={piso.values}")
        next(inputs)
    assert actual_outputs == expected_outputs
    
check(PISO10, DefinePISO(10)(), 20, inputs_generator(message))

print("Passed!")

Passed!


In [3]:
!magma -o coreir -m coreir -t PISO build/silica_piso.py
!coreir -i build/silica_piso.json -o build/silica_piso.v
!yosys -p 'synth_ice40 -top PISO -blif build/silica_piso.blif' build/silica_piso.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_piso.txt build/silica_piso.blif
!icetime -tmd hx1k build/silica_piso.txt  | grep -B 2 "Total path delay"

In Run Generators
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!

Modified?: Yes
2.27. Printing statistics.

=== PISO ===

   Number of wires:                265
   Number of wire bits:            552
   Number of public wires:         265
   Number of public wire bits:     552
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 20
     SB_DFF                         10
     SB_LUT4                        10

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/silica_piso.blif

In [9]:
import os
os.environ["MANTLE"] = "lattice"
from magma import compile
from mantle import DefinePISO as MantleDefinePISO

compile("build/mantle_piso", MantleDefinePISO(10))


check(MantleDefinePISO(10), DefinePISO(10)(), 20, inputs_generator(message))
print("Passed!")

compiling Mux2
compiling Register10
compiling PISO10
Passed!


In [10]:
!yosys -p 'synth_ice40 -top PISO10 -blif build/mantle_piso.blif' build/mantle_piso.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/mantle_piso.txt build/mantle_piso.blif
!icetime -tmd hx1k build/mantle_piso.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== PISO10 ===

   Number of wires:                 69
   Number of wire bits:            115
   Number of public wires:          69
   Number of public wire bits:     115
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 20
     SB_DFF                         10
     SB_LUT4                        10

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/mantle_piso.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          14 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          10 / 1280
  DFF        10
  CARRY      0
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted CLK$2, 10 / 10
  promoted 1 nets
    1 clk
  1 